In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

import jsonlines 
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
# dictionary_paths, create pattens 
from spacy.pipeline import EntityRuler

Diseases = '/nfs/gns/literature/lit-textmining-pipelines/automata/DiseaseDictionary.mwt' #(# of terms - 59088)
Genes_Proteins = '/nfs/gns/literature/lit-textmining-pipelines/automata/swissprot_Sept2014.2.3.mwt' #(# of terms - 347929)
Organism = '/nfs/gns/literature/lit-textmining-pipelines/automata/Organisms150507.2.mwt' #(# of terms - 1683021)

patterns = []

with open(Genes_Proteins, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    print('Adding Genes Dictionary')
    pattern_types = soup.find_all('t')
    for pattern in tqdm(pattern_types):
        patterns.append({"label": "GP", "pattern": pattern.text})


with open(Diseases, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    print('Adding Disease Dictionary')
    pattern_types = soup.find_all('t')
    for pattern in tqdm(pattern_types):
        patterns.append({"label": "DS", "pattern": pattern.text})
        
        
# with open(Organism, 'r') as f:
#     contents = f.read()
#     soup = BeautifulSoup(contents, 'lxml')
#     print('Adding Organisms Dictionary')
#     pattern_types = soup.find_all('r')
#     for pattern in tqdm(pattern_types):
#         patterns.append({"label": "OG", "pattern": [{"TEXT": {"REGEX": pattern.text.replace('[^A-Za-z0-9]','')}}]})

Adding Genes Dictionary


 41%|████      | 24177/59088 [00:00<00:00, 241758.70it/s]

Adding Disease Dictionary


100%|██████████| 59088/59088 [00:00<00:00, 242864.99it/s]


In [3]:
## load the best model
best_model_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best'
print("Loading from", best_model_path)
nlp = util.load_model_from_path(best_model_path)


Loading from /nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best


In [4]:
from spacy.pipeline import EntityRuler
rulerDictionary = EntityRuler(nlp, overwrite_ents=True)


In [ ]:
rulerDictionary.add_patterns(patterns)


In [ ]:
nlp.remove_pipe('entity_ruler')
nlp.add_pipe(rulerDictionary, after="ner")
print(nlp.pipeline)

In [ ]:
text = 'Interspecific tumor ZBTB12 PT Sergef Ribonuclease 5 EN116 overgrowth TGN48 Uncharacterized protein Rv2292c phenocopies the s.Crocodylurus Transmembrane protein 2 tarahumaras-frogs Putative HLA class I histocompatibility antigen, alpha chain H female gametophytic mutants feronia/sirène (fer/srn), lorelei (lre), nortia (nta), Turan (tun), evAn (evn), and Zea mays embryo sac 4 (ZmES4) RNAi-lines1314151617181920, PtdInsTP β,  hmg2l1, MXL8.8 which are defective in the reception of intraspecific PTs. GTP'
# text = 'These ractory cytopenia with unilineage dysplasia iris neoplasms results indicate the dentate gyrus is breast tumor mostly comprised of mature neurons (NeuN), along with a smaller population of precursor cells (nestin) and newly differentiated neurons (DCX), which corresponds to prior findings examining the relative number of each cell population in the dentate gyrus, indicating the proportion of cells labeled by the sensor approximately reflects physiological proportions [21, 22]. '
text = ' However, if a pollen grain originating from a different species (interspecific pollination) is placed on a plant\'s stigma, all the communication processes desc     27 ribed above have the potential to act as pre-zygotic post-pollination barriers.'
# text = 'Residual erythrocytes were removed by hypotonic lysis'
sentence =nlp(text)

print(sentence)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~ **** ~~~~~~~~~~~~~~~~~~~~~~~')
for ent in sentence.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)